In [1]:
from model.tasks import Task
from model.projects import StandardProject
import ipywidgets as widgets
import matplotlib.pyplot as plt

p = StandardProject()


def draw_task(task: Task):
    fig1, ax1 = plt.subplots()
    emissions = [s.get_co2_impact() for s in task.subtasks]
    names = [s.name for s in task.subtasks]
    ax1.pie(emissions, labels=names)
    ax1.set_title(task.name)


def draw_tasks(task: Task):
    if task.subtasks:
        draw_task(task)
        for subtask in task.subtasks:
            draw_tasks(subtask)


slider_style = {"description_width": "initial"}


@widgets.interact(
    dev_days=widgets.IntSlider(
        min=0,
        max=3000,
        value=p.dev_days,
        description="Dev man-days",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    design_days=widgets.IntSlider(
        min=0,
        max=500,
        value=p.design_days,
        description="Design man-days",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    spec_days=widgets.IntSlider(
        min=0,
        max=300,
        value=p.spec_days,
        description="Specifications man-days",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    management_days=widgets.IntSlider(
        min=0,
        max=3000,
        value=p.management_days,
        description="Management man-days",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    maintenance_days=widgets.IntSlider(
        min=0,
        max=1500,
        value=p.maintenance_days,
        description="Maintenance man-days",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    electricity_mix=widgets.FloatSlider(
        min=0,
        max=0.8,
        value=p.electricity_mix,
        description="Electricity mix (kg)",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    pue=widgets.FloatSlider(
        min=1,
        max=2.5,
        value=p.pue,
        description="PUE",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    servers_count=widgets.IntSlider(
        min=0,
        max=50,
        value=p.servers_count,
        description="Servers count",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    storage_tb=widgets.IntSlider(
        min=0,
        max=100,
        value=p.storage_size,
        description="Tb reserved",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    run_duration=widgets.IntSlider(
        min=0,
        max=3000,
        value=p.run_duration,
        description="Run days",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    avg_user_day=widgets.IntSlider(
        min=0,
        max=3000,
        value=p.avg_user,
        description="User per day",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    avg_user_minutes=widgets.IntSlider(
        min=0,
        max=250,
        value=p.avg_user,
        description="Average user time(minutes)",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
    avg_user_data=widgets.FloatSlider(
        min=0,
        max=20,
        value=p.avg_data,
        description="Average data transferred",
        style=slider_style,
        layout=widgets.Layout(width="400px"),
    ),
)
def draw(
    dev_days,
    design_days,
    spec_days,
    management_days,
    maintenance_days,
    electricity_mix,
    pue,
    servers_count,
    storage_tb,
    run_duration,
    avg_user_day,
    avg_user_minutes,
    avg_user_data,
):
    p.dev_days = dev_days
    p.design_days = design_days
    p.spec_days = spec_days
    p.management_days = management_days
    p.maintenance_days = maintenance_days
    p.electricity_mix = electricity_mix
    p.pue = pue
    p.servers_count = servers_count
    p.storage_size = storage_tb
    p.run_duration = run_duration
    p.avg_user = avg_user_day
    p.avg_time = avg_user_minutes
    p.avg_data = avg_user_data
    draw_tasks(p.root_task)

interactive(children=(IntSlider(value=2000, description='Dev man-days', layout=Layout(width='400px'), max=3000…